# 사용자 리뷰 분류하기

In [1]:
import pandas as pd
from numpy import array, asarray, zeros

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

##  실습 데이터

In [2]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [3]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


## 데이터 전처리

In [4]:
# 문장과 카테고리를 리스트에 저장
docs = df['paragraph'].tolist()
categories = df['category'].tolist()

In [5]:
# 토큰나이저로 단어들을 정수로 인코딩힙니다.
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)


def get_max_length(data):
    '''
    데이터에서 가장 긴 문장의 단어 개수를 리턴한다.
    '''
    max_length = 0
    for row in data:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

    
max_length = get_max_length(df['paragraph'])
print (max_length)

# 모든 문장에 패딩을 넣어 가장 긴 문장과 같은 길이의 문장으로 만듭니다. 
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

91


In [6]:
# 정수로 인코딩된 첫번째 문장입니다.
padded_docs[0]

array([162,  16, 163,   5, 164, 165,  56,  16, 166, 167,   1, 168,   9,
         6,  91,  25,  92,   2, 169,  40, 170, 171,  57,  26, 172,   3,
        12,  41,   2,  23,  58,   8, 173,  15,   4, 174,  10,  13,  59,
        93,  17,  12, 175, 176, 177, 178, 179,  94,  42,   1, 180,   2,
        95,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [7]:
# 카테고리를 인코딩합니다.

def category_encode(category):
    '''
    분류항목(food, sports) 역시 수치로 변경해야 합니다.
    '''
    if category == 'food':
        return [1,0]
    else:
        return [0,1]

labels = array([category_encode(category) for category in categories])

In [8]:
print("first document's label:",labels[0])

first document's label: [1 0]


### Glove 다운로드

In [9]:
# Glove 다운로드
# !wget -P glove "http://nlp.stanford.edu/data/glove.6B.zip"

In [10]:
# 압축 풀기
# !unzip glove.6B.zip

In [11]:
# 내려받은 Glove 벡터를 딕셔너리로 불려온다.
embeddings_index = dict()
f = open('./glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    
    # 100개의 값을 가지는 array로 변환
    word_vector_arr = asarray(values[1:], dtype='float32')
    embeddings_index[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embeddings_index))

400000개의 Embedding vector가 있습니다.


In [12]:
# Glove 벡터로 실습 문장들의 단어를 인코딩힙니다.

# 풀고자 하는 문제의 단어 집합 크기의 행과 100개의 열을 가지는 행렬을 생성합니다.
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## 모델생성(임베딩 유지)

In [13]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
# 모델의 손실 함수 및 옵티마이저를 정합니다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 학습
model.fit(padded_docs, array(labels), epochs=5, verbose=1)

Metal device set to: Apple M2
Epoch 1/5


2023-11-06 23:15:55.906244: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-06 23:15:55.906526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-11-06 23:15:56.065977: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 311ms/step - loss: 0.6871 - accuracy: 0.7000
Epoch 2/5
1/1 [==============================] - 0s 8ms/step - loss: 0.4719 - accuracy: 0.6500
Epoch 3/5
1/1 [==============================] - 0s 8ms/step - loss: 0.2044 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 8ms/step - loss: 0.0969 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 9ms/step - loss: 0.0690 - accuracy: 1.0000


2023-11-06 23:15:56.274707: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## 모델생성(임베딩 튜닝)

In [14]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True)
model.add(e)
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
# 모델의 손실 함수 및 옵티마이저를 정합니다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 학습
model.fit(padded_docs, array(labels), epochs=5, verbose=1)

Epoch 1/5
1/1 [==============================] - 0s 251ms/step - loss: 0.6871 - accuracy: 0.7000
Epoch 2/5
1/1 [==============================] - 0s 12ms/step - loss: 0.4573 - accuracy: 0.6500
Epoch 3/5
1/1 [==============================] - 0s 14ms/step - loss: 0.1894 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 14ms/step - loss: 0.0876 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 12ms/step - loss: 0.0609 - accuracy: 1.0000


2023-11-06 23:15:56.604678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
